In [1]:
from typing_extensions import TypedDict

class State(TypedDict):
    topic: str
    draft: str
    feedback: str


In [2]:
import os
from dotenv import load_dotenv

load_dotenv()
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

In [3]:
from langchain_groq import ChatGroq
llm=ChatGroq(model="llama-3.1-8b-instant")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000021934936690>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000021934937830>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
#first node to collect data 
def collect_data(state: State):
    print("Collecting data...")
    topic = state["topic"]
    return {"topic": topic, "draft": "", "feedback": ""}


# second node to write the draft
def write_material(state: State):
    print("Writing material...")
    topic = state["topic"]
    draft = llm.invoke(f"Write a blog post draft about: {topic}")
    return {"draft": draft}

# feedback loop if given any feedback will rewrite the post 
def review_material(state: State):
    print("Reviewing material...")
    draft = state["draft"]
    feedback = state.get("feedback", "")

    if feedback:
        revised = llm.invoke(f"Rewrite this draft based on feedback: {feedback}\n\nDraft:\n{draft}")
        return {"draft": revised}
    else:
        return {"draft": draft}




In [5]:
from langgraph.graph import StateGraph, START, END

builder = StateGraph(State)

builder.add_node("collect_data", collect_data)
builder.add_node("write_material", write_material)
builder.add_node("review_material", review_material)

builder.add_edge(START, "collect_data")
builder.add_edge("collect_data", "write_material")
builder.add_edge("write_material", "review_material")
builder.add_edge("review_material", END)

graph = builder.compile()


In [6]:
def run_post_writer(topic: str, feedback: str = ""):
    initial_state = {"topic": topic, "draft": "", "feedback": feedback}
    result = graph.invoke(initial_state)

    print("\n" + "="*50)
    print(f"📌 Topic: {topic}")
    print("="*50)
    print("📝 Draft:\n")
    print(result["draft"])
    print("="*50 + "\n")

while True:
    topic = input("Enter a topic for your post: ")
    if topic.lower() in ["quit", "exit", "q"]:
        print("Goodbye!")
        break

    # Step 1: Generate and show draft
    run_post_writer(topic)

    # Step 2: Ask for feedback
    feedback = input("Do you have feedback for revision? (leave empty if none): ")
    if feedback.strip():
        run_post_writer(topic, feedback=feedback)


Writing material...
Reviewing material...

📌 Topic: write me a post on japanese ghost Ubume
📝 Draft:

content='**The Ubume: A Haunting Figure from Japanese Folklore**\n\nIn Japan, a vast array of supernatural creatures have captured the imagination of the people, each with their unique characteristics and stories. Among these is the Ubume, a ghostly figure shrouded in mystery and terror. This enigmatic creature has been a part of Japanese folklore for centuries, and its legend continues to fascinate and intrigue even today.\n\n**Origins and Characteristics**\n\nThe Ubume is a yurei, a type of vengeful spirit found in Japanese folklore. It is said to be the ghost of a woman who has died during childbirth, often as a result of a tragic or untimely death. The Ubume is characterized by its pale, ghostly appearance, with long black hair and a white kimono. Its face is often described as being gaunt and skeletal, with sunken eyes that seem to bore into the souls of those who cross its path.\